In [1]:
import torch
import torch.nn as nn
import numpy as np



In [29]:

alpha = 0.1


def fourier_series(n):
    # n even
    return -480/(np.pi**4*(n**4))


def heat_function(x, t: int):
    a_0 = 1/3
    sum = 0
    for i in range(1, 20):
        exponential = np.exp(-1*alpha*(2*i*np.pi)**2*t)
        sum += fourier_series(2*i)*np.cos(np.pi*2*i*x)*exponential
    return a_0 + sum


def compute_residual(model, x, t):

    x = x.clone().detach().requires_grad_(True)
    t = t.clone().detach().requires_grad_(True)

    u = model(x, t)
    ones = torch.ones_like(u)
    d_t = torch.autograd.grad(
        u, t, grad_outputs=ones, create_graph=True, retain_graph=True
        )[0]
    d_x = torch.autograd.grad(
        u, x, grad_outputs=ones, create_graph=True, retain_graph=True
    )[0]
    d_xx = torch.autograd.grad(
        d_x, x, grad_outputs=torch.ones_like(d_x), create_graph=True,
        retain_graph=True
    )[0]

    return d_t - alpha*d_xx


def initial_condition(x):
    return 10*(x-x**2)**2 + 3


class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        layer = [nn.Linear(2, 100), nn.ReLU()]
        for i in range(8):
            layer += [nn.Linear(100, 100), nn.ReLU()]
        layer += [nn.Linear(100, 1), nn.ReLU()]
        self.net = nn.Sequential(*layer)

    def forward(self, x, t):
        inp = torch.cat([x, t], dim=1)
        return self.net(inp)


In [30]:
num_epochs=5000
num_collocation_res=1000
num_collocation_ic=500
num_collocation_bc=600
lr=1e-3
lambda_residual=10.0
lambda_ic=6.0
lambda_bc=5.0


x_col_res = torch.empty(num_collocation_res, 1).uniform_(0, 1)
t_col_res = torch.empty(num_collocation_res, 1).uniform_(0, 1)


In [4]:
x_col_res = torch.rand(num_collocation_res, 1)
t_col_res = torch.rand(num_collocation_res, 1)

print(x_col_res)

tensor([[7.5928e-01],
        [7.8389e-01],
        [9.2870e-01],
        [3.0847e-01],
        [4.6604e-01],
        [1.7327e-01],
        [7.9045e-01],
        [2.0783e-02],
        [4.1064e-01],
        [6.7615e-01],
        [3.5346e-01],
        [5.1663e-01],
        [4.3682e-01],
        [9.2645e-01],
        [5.5484e-01],
        [4.2119e-01],
        [9.9443e-02],
        [6.9898e-01],
        [7.5395e-01],
        [5.4140e-02],
        [7.0713e-01],
        [4.4426e-01],
        [7.1191e-01],
        [3.5726e-01],
        [2.9819e-01],
        [2.3594e-01],
        [9.1517e-01],
        [1.1735e-01],
        [7.7984e-01],
        [2.4695e-01],
        [3.6111e-01],
        [3.3513e-02],
        [5.4610e-01],
        [7.2393e-01],
        [3.4262e-01],
        [8.0528e-01],
        [4.9904e-01],
        [6.4455e-01],
        [2.9404e-01],
        [3.7845e-01],
        [4.7098e-01],
        [3.9407e-01],
        [6.9496e-01],
        [5.1788e-01],
        [9.9673e-01],
        [1

In [36]:


def train_pinn(
        num_epochs=5000,
        num_collocation_res=1000,
        num_collocation_ic=500,
        num_collocation_bc=600,
        lr=1e-3,
        lambda_residual=10.0,
        lambda_ic=6.0,
        lambda_bc=5.0
):
    model = NeuralNetwork()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)

    # Residual Collocation
    x_col_res = torch.empty(num_collocation_res, 1).uniform_(0, 1)
    t_col_res = torch.empty(num_collocation_res, 1).uniform_(0, 1)

    # Initial Condition Collocation
    x_col_ic = torch.empty(num_collocation_ic, 1).uniform_(0, 1)
    t_col_ic = torch.zeros((num_collocation_ic, 1))

    # Boundary Condition Collocation
    t_x_bc = torch.empty(num_collocation_bc, 1).uniform_(0, 1)
    x_bc = torch.zeros((num_collocation_bc, 1), requires_grad=True)
    t_l_bc = torch.empty(num_collocation_bc, 1).uniform_(0, 1)
    l_bc = torch.ones((num_collocation_bc, 1), requires_grad=True)

    # Neumann
    ux_0_bc = torch.zeros((num_collocation_bc, 1))
    ux_1_bc = torch.zeros((num_collocation_bc, 1))

    for _ in range(1000):
        optimizer.zero_grad()
        residual = compute_residual(model, x_col_res, t_col_res)
        loss_residual = torch.mean(residual**2)
        model_ic = model(x_col_ic, t_col_ic)
        #for i in range(10):
        #    print(model_ic[i], end="_THEN_")
        #    print(initial_condition(x_col_ic)[i])
        loss_ic = torch.mean((model_ic-initial_condition(x_col_ic))**2)
        print(loss_ic, _)
        loss = loss_ic
        loss.backward()
        optimizer.step()
        """
        # Boundary
        u_0_bc = model(x_bc, t_x_bc)
        du_0_bc = torch.autograd.grad(
            u_0_bc, x_bc, grad_outputs=torch.ones_like(u_0_bc),
            create_graph=True
        )[0]

        u_l_bc = model(l_bc, t_l_bc)
        du_l_bc = torch.autograd.grad(
            u_l_bc, l_bc, grad_outputs=torch.ones_like(u_l_bc),
            create_graph=True
        )[0]

        loss_0_bc = torch.mean((du_0_bc-ux_0_bc)**2)
        loss_1_bc = torch.mean((du_l_bc-ux_1_bc)**2)
        loss_b = (loss_0_bc + loss_1_bc)
        """
    save_path = "parametersheat.pth"
    torch.save(
            {'model_state_dict': model.state_dict()}, save_path
        )
    return model




In [37]:
model = train_pinn()

tensor(10.5638, grad_fn=<MeanBackward0>) 0
tensor(10.4369, grad_fn=<MeanBackward0>) 1
tensor(10.3104, grad_fn=<MeanBackward0>) 2
tensor(10.1846, grad_fn=<MeanBackward0>) 3
tensor(10.0547, grad_fn=<MeanBackward0>) 4
tensor(9.9202, grad_fn=<MeanBackward0>) 5
tensor(9.7847, grad_fn=<MeanBackward0>) 6
tensor(9.6466, grad_fn=<MeanBackward0>) 7
tensor(9.4983, grad_fn=<MeanBackward0>) 8
tensor(9.3343, grad_fn=<MeanBackward0>) 9
tensor(9.1508, grad_fn=<MeanBackward0>) 10
tensor(8.9448, grad_fn=<MeanBackward0>) 11
tensor(8.7127, grad_fn=<MeanBackward0>) 12
tensor(8.4344, grad_fn=<MeanBackward0>) 13
tensor(8.0970, grad_fn=<MeanBackward0>) 14
tensor(7.6909, grad_fn=<MeanBackward0>) 15
tensor(7.1990, grad_fn=<MeanBackward0>) 16
tensor(6.5909, grad_fn=<MeanBackward0>) 17
tensor(5.8273, grad_fn=<MeanBackward0>) 18
tensor(4.8747, grad_fn=<MeanBackward0>) 19
tensor(3.7219, grad_fn=<MeanBackward0>) 20
tensor(2.3989, grad_fn=<MeanBackward0>) 21
tensor(1.0659, grad_fn=<MeanBackward0>) 22
tensor(0.1543, g

KeyboardInterrupt: 